#### Собирает данные о геопозиции

In [50]:

import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic


In [ ]:
#Для каждого уникального адреса собирает геопозицию и полный адрес


df = pd.read_csv('total.csv')
df['address'] = df['street'] + ' ' + df['house'].str.replace('\xa0', '')
a = df.address.unique()


geolocator = Nominatim(user_agent="housing", timeout=20)


d = {'address':[], 'latitude':[], 'longitude':[], 'full_address':[]}
count = 0
for i in a:
    d['address'].append(i)
    i = 'россия екатеринбург' + ' ' + i
    location = geolocator.geocode(i)

#Для некоторых адресов геопозиция не определяется
#удаляет последний символ в адресе, повторно определение геопозиции
    
    while hasattr(location, 'latitude') is False:
        i = i[:i.rfind(' ')]
        location = geolocator.geocode(i)
    try:
    
        d['latitude'].append(location.latitude)
        d['longitude'].append(location.longitude)
        d['full_address'].append(location.address)
    except:
        
        d['latitude'].append(0)
        d['longitude'].append(0)
        d['full_address'].append(location.address)
    count+=1
    if count%100 == 0:
        print(count)


    

In [38]:
#координаты центра екатеринбурга

c = (56.83725377741802,60.597350913273274)



#координаты метро
square_1905 = (56.8362716853893,60.59968979953426)
dinamo = (56.84782181862475,60.59947472070306)
uralskaya = (56.857638533270126,60.60010772203058)
mash = (56.87755339342598,60.6116150117623)
uralmash = (56.88860434388676,60.61365789683971)
kosmos = (56.901832944294,60.613943098293326)
geolog = (56.827545099355305,60.60260559413865)
chkalov = (56.80774218566658,60.61050189281998)
botan = 56.79736609518068,60.63237728844232

metro_list = (square_1905, dinamo, uralskaya, mash, uralmash, kosmos, geolog, chkalov, botan)



In [ ]:
df1 = pd.DataFrame(d)

In [203]:
#расчитывает расстояние до центра для каждого адреса

df1['distance_centre'] = 0

for i in range(len(df1)):

    df1['distance_centre'][i] = round(geodesic(c, (float(df1.latitude[i]), float(df1.longitude[i]))).kilometers, 1)

/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_6082/1676684233.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['distance_centre'][i] = round(geodesic(c, (float(df1.latitude[i]), float(df1.longitude[i]))).kilometers, 1)


In [204]:
#расчитывает расстояние до ближайшей станции метро

df1['distance_metro'] = 0
yo = []
for i in range(len(df1)):
    a = []
    for j in metro_list:
        
        a.append(geodesic(j, (float(df1.latitude[i]), float(df1.longitude[i]))).kilometers)
        df1['distance_metro'][i] = round(min(a), 1)

    

/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_6082/48496119.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['distance_metro'][i] = round(min(a), 1)


In [226]:
#достает район из fulladdress

df1['area'] = 0
j = 0

for i in list(df1.full_address):

    df1['area'][j] = i[i[:i.find(' район')].rfind(' ')+1:i.find(' район')]
    j+=1

/var/folders/m9/x7jxk9jx2g9ch4ngnwz3n8wh0000gn/T/ipykernel_6082/3926049492.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['area'][j] = i[i[:i.find(' район')].rfind(' ')+1:i.find(' район')]


In [251]:
df1.to_csv('coords.csv')
